In [2]:
import numpy as np

In [1]:

def postmean(g_hat, g_bar, n, d_star, t2):
    return (t2*n*g_hat+d_star * g_bar) / (t2*n+d_star)

def postvar(sum2, n, a, b):
    return (0.5 * sum2 + b) / (n / 2.0 + a - 1.0)

In [3]:

def it_sol(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)

        change = max((abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max())
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (g_new, d_new)
    return adjust 

In [4]:
def aprior(gamma_hat):
    m = np.mean(gamma_hat)
    s2 = np.var(gamma_hat,ddof=1)
    return (2 * s2 +m**2) / float(s2) 

def bprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = np.var(gamma_hat,ddof=1)
    return (m*s2+m**3)/s2

In [ ]:
y_1f1=np.array([10,12,13,11.2,16])
y_1f2=np.array([110,112,113,111.2,116])

g_hat_f1 = np.mean(y_1f1)
d_hat_f1 = np.var(y_1f1)

g_hat_f2 = np.mean(y_1f2)
d_hat_f2 = np.var(y_1f2)

g_bar = np.mean([g_hat_f1,g_hat_f2])
t2 = np.var([g_hat_f1,g_hat_f2])**2

a_prior = list(map(aprior, delta_hat))
b_prior = list(map(bprior, delta_hat))

adjust = it_sol(y_1f1, d_hat_f1, g_bar, t2)